In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!pip install ffmpeg
!pip3 install face_recognition

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6084 sha256=87666d2b00b529ddbd2a60a6d7338a8c6f452239b4fa6e04dcd0b0be17e6eab7
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
     |████████████████████████████████| 100.2MB 73kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=3036e79ef1331d59ef25a89b090bf4958960245b631c6d7d046065a9456fe830
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [2]:
!pip install -U imageio==2.6.1

     |████████████████████████████████| 3.3MB 2.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [7]:
import os
os.listdir("drive/My Drive/MesoNet")

['Meso4_F2F',
 'MesoInception_F2F',
 'MesoInception_DF',
 'Meso4_DF',
 'pipeline.py',
 'classifiers.py']

In [0]:
import sys
sys.path.insert(0,"drive/My Drive/MesoNet/")

In [9]:
test_dir = "drive/My Drive/DeepFake_test/"

test_videos = sorted([x for x in os.listdir(test_dir) if x[-4:] == ".mp4"])
len(test_videos)

5

In [15]:
import numpy as np
from classifiers import *
from pipeline import *
import time

Using TensorFlow backend.


Load Pretrained Weights

In [0]:
classifier = Meso4()
classifier.load('drive/My Drive/MesoNet/Meso4_F2F')

Predict Individual Video

In [0]:
def predict_on_video(classifier, vid, dirname, frame_subsample_count = 30):
    '''
    params : classifier
             vid = video filename
             dirname = video file directory
    '''
    try:
        print('Dealing with video ', vid)

        # Compute face locations and store them in the face finder
        start_time = time.time()
        face_finder = FaceFinder(join(dirname, vid), load_first_face = False)
        skipstep = max(floor(face_finder.length / frame_subsample_count), 0)
        face_finder.find_faces(resize=0.5, skipstep = skipstep)
        elapsed = time.time() - start_time
        print("Found Face %f sec." % (elapsed))
        
        print('Predicting ', vid)
        start_time = time.time()
        gen = FaceBatchGenerator(face_finder)
        elapsed = time.time() - start_time
        print("Generated Batch %f sec." % (elapsed))
        
        start_time = time.time()
        p = predict_faces(gen, classifier)
        elapsed = time.time() - start_time
        print("Prediction %f sec." % (elapsed))

        prediction = (np.mean(p > 0.5), p)
        
        return prediction
    
    except Exception as e:
        print("Prediction error on video %s: %s" % (vid, str(e)))
    
    return 0.5    

Compute accuracy of entire dataset

In [0]:
def compute_accuracy(classifier, dirname, frame_subsample_count = 30):
    '''
    params: classifier
            dirname = video directory
    
    '''
    filenames = [f for f in listdir(dirname) if isfile(join(dirname, f)) and ((f[-4:] == '.mp4') or (f[-4:] == '.avi') or (f[-4:] == '.mov'))]
    predictions = {}
    
    
    for vid in filenames[:5]:
        start_time = time.time()
        pred = predict_on_video(classifier, vid, dirname, frame_subsample_count)
        predictions[vid] = pred
        elapsed = time.time() - start_time
        print("Elapsed %f sec." % (elapsed))
    return predictions

In [0]:
'''
Running each video on a single core with threads
'''
# from concurrent.futures import ThreadPoolExecutor

# def predict_on_video_set(videos, num_workers):
#     def process_file(i):
#         filename = videos[i]
#         y_pred = predict_on_video(os.path.join(test_dir, filename), batch_size=frames_per_video)
#         return y_pred

#     with ThreadPoolExecutor(max_workers=num_workers) as ex:
#         predictions = ex.map(process_file, range(len(videos)))

#     return list(predictions)

In [24]:
predictions = compute_accuracy(classifier, test_dir)
result = {}
for video_name in predictions:
    result[video_name] = predictions[video_name][0]
    print('`{}` video class prediction :'.format(video_name), predictions[video_name][0])

Dealing with video  aelfnikyqj.mp4
Face extraction warning :  0 - found face in full frame [(83, 1011, 194, 899)]
Face extraction report of not_found : 1
Face extraction report of no_face : 0
Predicting  aelfnikyqj.mp4
Elapsed 18.163348 sec
Dealing with video  abarnvbtwb.mp4
Face extraction warning :  0 - found face in full frame [(208, 999, 401, 807)]
Face extraction report of not_found : 1
Face extraction report of no_face : 0
Predicting  abarnvbtwb.mp4
Elapsed 19.683799 sec
`aelfnikyqj.mp4` video class prediction : 1.0
`abarnvbtwb.mp4` video class prediction : 1.0


In [0]:
import pandas as pd
submission_df = pd.DataFrame(list(result.items()), columns={'filename','label'})
submission_df.head()